In [ ]:
import pandas as pd
import numpy as np

df_heart=pd.read_csv('Heart_Modul2.csv')
df_heart.head(20)

In [ ]:
df_heart2=df_heart.drop('Name',axis=1)
df_heart2.head(50)

In [ ]:
df_heart2['ChestPainType' ].value_counts()

In [ ]:
print("data null \n",df_heart2.isnull().sum())
print("\ndata kosong \n",df_heart2.empty)
print("\ndata nan \n",df_heart2.isna().sum())

In [ ]:
median_chole = df_heart2['Cholesterol'].median()

print(median_chole)

df_heart2['Cholesterol' ] = df_heart2['Cholesterol' ].fillna(median_chole)

In [ ]:
print(df_heart2['Cholesterol' ].value_counts())

print("data null \n",df_heart2.isnull().sum())
print("\ndata kosong \n",df_heart2.empty)
print("\ndata nan \n",df_heart2.isna().sum())

In [ ]:
print("Sebelum drop missing value",df_heart2.shape)
df_heart2 = df_heart2.dropna(how="any",inplace=False)
print("Sesudah drop missing value" ,df_heart2.shape)

In [ ]:
df_heart2['Sex'].value_counts()

In [ ]:
print("Sebelum drop data dengan gender Bi ",df_heart2.shape)
df_heart2=df_heart2[df_heart2['Sex'] != 'Bi' ]
print("Sesudah drop data dengan gender Bi ", df_heart2.shape)

In [ ]:
df_heart2.head(20)

In [ ]:
print("Sebelum Pengecekan data duplikat, ", df_heart2.shape)
df_heart3=df_heart2.drop_duplicates(keep='last')
print("Setelah Pengecekan data duplikat, ", df_heart3.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x = df_heart3.drop(columns=['HeartDisease'],axis=1)
y=df_heart3['HeartDisease' ]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=79)

print(x_train.shape)
print(x_test.shape)

In [13]:
from sklearn. preprocessing import OneHotEncoder
from sklearn. compose import make_column_transformer

kolom_kategori=['Sex','ChestPainType', 'RestingECG','ExerciseAngina','ST_Slope']

transform = make_column_transformer(
(OneHotEncoder(), kolom_kategori), remainder='passthrough'

)

In [ ]:
x_train_enc=transform.fit_transform(x_train)
x_test_enc=transform. fit_transform(x_test)

df_train_enc=pd.DataFrame(x_train_enc,columns=transform.get_feature_names_out())
df_test_enc=pd. DataFrame(x_test_enc, columns=transform.get_feature_names_out())

df_train_enc.head(10)
df_test_enc.head(10)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectPercentile ,SelectKBest
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

pipe_svm = Pipeline(steps=[
    ('scale', MinMaxScaler()),
    ('feat_select', SelectKBest()),
    ('clf', SVC(class_weight='balanced'))
])

params_grid_svm = [
    {
        'scale': [MinMaxScaler()],
        'feat_select__k':np.arange(2,6),
        'clf__kernel': ['poly','rbf'],
        'clf__C':[0.1,1],
        'clf__gamma':[0.1, 1]

    },
    {
        'scale': [MinMaxScaler()],
        'feat_select':[SelectPercentile()],
        'feat_select__percentile':np.arange(20,50),
        'clf__kernel': ['poly', 'rbf'],
        'clf__C':[ 0.1, 1],
        'clf__gamma': [0.1, 1]
    },
    {
        'scale': [StandardScaler()],
        'feat_select__k':np.arange(2,6),
        'clf__kernel': ['poly','rbf'],
        'clf__C':[0.1, 1],
        'clf__gamma':[0.1, 1]
    },
    {
        'scale': [StandardScaler()],
        'feat_select': [SelectPercentile()],
        'feat_select__percentile':np.arange(20,50),
        'clf__kernel': ['poly', 'rbf'],
        'clf__C':[0.1, 1],
        'clf__gamma': [0.1, 1]
    }
]

estimator_svm = Pipeline(pipe_svm)

SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=79)

GSCV_SVM = GridSearchCV(pipe_svm, params_grid_svm, cv=SKF)

GSCV_SVM.fit(x_train_enc, y_train)
print("GSCV training finished")

In [ ]:
print("CV Score : {}".format(GSCV_SVM.best_score_))

print("Test Score: {}".format(GSCV_SVM.best_estimator_.score(x_test_enc, y_test)))

print("Best model:", GSCV_SVM.best_estimator_)
mask = GSCV_SVM.best_estimator_.named_steps['feat_select' ].get_support()
print("Best features:", df_train_enc.columns[mask])


SVM_pred = GSCV_SVM.predict(x_test_enc)

import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, SVM_pred, labels=GSCV_SVM. classes_)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=GSCV_SVM.classes_)
disp.plot()
plt.title("SVM Confusion Matrix")
plt.show()


print("Classification report SVM:\n", classification_report(y_test, SVM_pred))

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import numpy as np

pipe_RF=[('data scaling', StandardScaler()),
         ('feature select', SelectKBest()),
         ('clf', RandomForestClassifier(random_state=79,class_weight='balanced'))]

params_grid_RF = [{
    'data scaling': [StandardScaler()],
    'feature select__k': np.arange(2, 6),
    'clf__max_depth': np.arange(4, 5),
    'clf__n_estimators': [100, 150]
},
{
    'data scaling': [StandardScaler()],
    'feature select': [SelectPercentile()],
    'feature select__percentile': np.arange(20, 50),
    'clf__max_depth': np.arange(4, 5),
    'clf__n_estimators': [100, 150]
},
{
    'data scaling': [MinMaxScaler()],
    'feature select__k': np.arange(2, 6),
    'clf__max_depth': np.arange(4,5),
    'clf__n_estimators': [100, 150]
},
{
    'data scaling': [MinMaxScaler()],
    'feature select': [SelectPercentile()],
    'feature select__percentile': np.arange(20, 50),
    'clf__max_depth': np.arange(4, 5),
    'clf__n_estimators': [100, 150]
}
]

estimator_RF = Pipeline(pipe_RF)
GSCV_RF=GridSearchCV(estimator_RF,params_grid_RF,cv=SKF)
GSCV_RF.fit(x_train_enc,y_train)

print("GSCV training finished")

In [ ]:
print("CV Score: {}".format(GSCV_RF.best_score_))
print("Test Score: {}".format(GSCV_RF.best_estimator_.score(x_test_enc, y_test)))
print("Best model:", GSCV_RF.best_estimator_)

mask = GSCV_RF.best_estimator_.named_steps['feature select' ].get_support()
print("Best features:", df_train_enc.columns[mask])


RF_pred = GSCV_RF.predict(x_test_enc)

import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, RF_pred, labels=GSCV_RF.classes_)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=GSCV_RF.classes_)
disp.plot()

plt.title("Random Forest Confusion Matrix")
plt.show()

print("Classification report RF: \n", classification_report(y_test, RF_pred))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier

pipe_GBT = Pipeline(steps=[
        ('feat_select',SelectKBest()),
        ('clf', GradientBoostingClassifier(random_state=79))])

params_grid_GBT=[
    {
        'feat_select__k': np.arange(2,6),
        'clf__max_depth': [*np.arange(4,5)],
        'clf__n_estimators':[100,150],
        'clf__learning_rate': [0.01,0.1,1]
    },
    {
        'feat_select':[SelectPercentile()],
        'feat_select__percentile':np.arange(20,50),
        'clf__max_depth': [*np.arange(4,5)],
        'clf__n_estimators': [100,150],
        'clf__learning_rate' : [0.01,0.1,1]
    },
    {
        'feat_select__k': np.arange(2,6),
        'clf__max_depth': [*np.arange(4,5)],
        'clf__n_estimators': [100,150],
        'clf__learning_rate': [0.01,0.1,1]
    },
    {
        'feat_select':[SelectPercentile()],
        'feat_select__percentile':np.arange(20,50),
        'clf__max_depth': [*np.arange(4,5)],
        'clf__n_estimators': [100,150],
        'clf__learning_rate':[0.01,0.1,1]
    }
]

GSCV_GBT = GridSearchCV(pipe_GBT,params_grid_GBT,cv=StratifiedKFold(n_splits=5))
GSCV_GBT.fit(x_train_enc,y_train)
print("GSCV Finished")

In [ ]:
print("CV Score : {}".format(GSCV_GBT.best_score_))
print("Test Score : {}".format(GSCV_GBT.best_estimator_.score(x_test_enc, y_test)))
print("Best model: ", GSCV_GBT.best_estimator_)

mask = GSCV_GBT.best_estimator_.named_steps['feat_select'].get_support()
print("Best features: ", df_train_enc.columns[mask])

RF_pred = GSCV_GBT.predict(x_test_enc)

import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, RF_pred, labels=GSCV_GBT.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=GSCV_GBT.classes_)
disp.plot()

plt.title("GBT Confusion Matrix")
plt.show

print("Classification report GBT: \n", classification_report(y_test, RF_pred))

In [ ]:
import pickle

with open('GBT_heartDisease_model.pkl','wb') as r:
    pickle.dump((GSCV_RF),r)

    print("Model GBT berhasil disimpan")